# **Plagiarism Detection Checker - Application of Levenshtein Distance.**
# **Done By:**

**1. DEV VORA – 1814120**

**2. JAINAM ZOBALIYA – 1814123**

**3. VAIBHAVI KUNDLE – 1814127**

**4. SHREYA KULKARNI – 1814129**

## Firstly, We have defined a function which will calculate the levenshtein distance of the Test and Source sentences.

In [ ]:
def leven_distance(s1, s2):
  m=len(s1)+1
  n=len(s2)+1
  mat = {}
  for i in range(m): 
    mat[i,0]=i
  for j in range(n): 
    mat[0,j]=j
  #print(tbl)
  for i in range(1, m):
    for j in range(1, n):
      if s1[i-1] == s2[j-1]:
        cost = 0 
      else:
        cost = 2
      mat[i,j] = min(mat[i, j-1]+1, mat[i-1, j]+1, mat[i-1,j-1]+cost)
  return mat[i,j]

## Before calculation of levenshtein distance , we will be removing the stopwords as a step for pre-processing the sentences.

In [ ]:
stopwords = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

In [ ]:
def rem_stop_words(line,stopwords):
  #convert line to lowercase, split wordwise
  line_tokens = line.lower().split()
  #make a list for the word not in the stopword list
  tokens_without_sw = [word for word in line_tokens if not word in stopwords]
  new_line = " ".join(tokens_without_sw)
  return new_line

## Here we have defined a function which will split the sentence line by line.
## We will be calculating the plagarism for each line. Accordingly the plagarism percentage will be calculated.
## This step can also be referred to as the tokenisation of sentences.

In [ ]:
def para_split(line):
  line_split = line.split(". ")
  #print(line_split)
  return line_split

# Here we define the code to covert the results to a pdf file.
# The font size,color and style is defined for the pdf file.

In [ ]:
!pip install fpdf
from fpdf import FPDF
from google.colab import files
result =""
plagArr = []
def makePDF(fileName):
  f = open(fileName, "r")
  pdf = FPDF()
  pdf.set_right_margin(100.0)
  pdf.page_no()
  pdf.set_auto_page_break(True)
  pdf.add_page()
  pdf.set_font("Times", "B" ,size = 30)
  pdf.cell(200, 10, txt = "***** Plagiarism Report *****", ln = 1, align = 'C')
  #set font for the body
  pdf.set_font("Times", size = 12)
  pdf.cell(200, 10, txt ="Note: Plagiarised Sentences are colored in red.", ln = 1, align = 'C')
  # effective_page_width = pdf.w - 3*pdf.l_margin
  for x in f:
      if x in plagArr:
        #set color for the plagiarised text
        pdf.set_text_color(255, 0, 0)
      if x == result:
        #set font size for result
        pdf.set_font("Times", "B" ,size = 18)
      pdf.multi_cell(200, 10, txt = x)
      pdf.set_text_color(0, 0, 0)
  plagArr.clear()
  pdf.output("Results.pdf")
  # files.download(fileName+'.pdf')

  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40722 sha256=a945cbbf8fe2da312688ffea7bedd1be2768d5b87c9cdf812d363e687deca417
  Stored in directory: /root/.cache/pip/wheels/9a/e9/77/4554ff5c99bc3f487c8d69620d8c41d99d54e9c54ab20ef4c9
Successfully built fpdf


## We will now be calling all the functions defined above for the pre-processing of the sentences. After which we will be calculating the levenshtein distance.
## Here, we are calculating the similarity by subtracting the ratio of Levenshtein distance and maximum length from the two strings from 1.We then multiply this value by 100 to get the similarity percentage.
## If this calculated similarity percentage is greater than 50% we consider the line as plagiarised.
## The number of sentences plagiarised is updated everytime the condition for the same is satisfied.
## According to the plagiarised number of lines, the plagiarism percentage is calculated.

In [ ]:
def plagiarism_checker(Test,Source):
  sentence = ""
  sentence_1 = ""
  sentence_2 = ""
  s1_1 = para_split(Test)
  for x in range(len(s1_1)):
    sentence_1 = sentence_1 +s1_1[x]
    if s1_1[x][-1] !=".":
      sentence_1 = sentence_1 + "."
    sentence_1 = sentence_1 + "\n"
  sentence_1 = "\n"+ sentence_1
  sentence = sentence_1 + "\n"
  count = 1
  plag_count = 0
  TestArr = []
  SourceArr = []
  new_Test = rem_stop_words(Test,stopwords)
  new_Source = rem_stop_words(Source,stopwords)
  for s in para_split(new_Test) :
    TestArr.append(s)
  for s in para_split(new_Source) :
    SourceArr.append(s)
  sentence2 = ""
  for s1,s2 in zip(TestArr,SourceArr) :
    ldist = leven_distance(s1,s2) 
    maxlen = max(len(s1),len(s2))
    print()
    print("s1 is: ",s1)
    print("s2 is: ",s2)
    print("Leven: ", ldist)
    print("Max Len: ", maxlen)
    percentage_similarity = (1- (ldist/maxlen)) * 100
    # print("PS: ",percentage_similarity)
    if percentage_similarity > 50 :
      s1New = s1_1[count-1]
      s1New = s1New + ".\n"
      plagArr.append(s1New)
      print("Sentence {} is plagiarised".format(count))
      plag_count = plag_count+1
    #sentences count  
    count = count+1
  global result
  if plag_count == 0:
    s4 = "The document is 100% unique"
    sentence3 = ""
    sentence3 = sentence3 + s4 
    result = sentence3 + "\n"
    print("The document is 100% unique")
  else:
    per = (plag_count/len(SourceArr)*100)
    s5 = "The document has been detected with a Plagiarised percentage of: "
    sentence3 = ""
    sentence3 = sentence3 + s5 + str(per)+"%"
    result = sentence3 +"\n"
    print("The document has {} % of plagiarism detected".format((per)))
  sentence = sentence + sentence3 + "\n"
  return sentence

In [ ]:
import os.path
from os import path
def menu():
  print("********WELCOME TO PLAGIARISM DETECTION TOOL**********\n")
  if (path.exists('Results.pdf')):
    !rm Results.pdf
  choice = int(input("""
                      1: Sentence plagiarism checker
                      2: Document plagiarism checker
                      3: Download plagiarism report and Quit
                      4: Quit!
                      Please enter your choice: """))
  if choice == 1:
    source = str(input("Enter the sentence to be checked: "))
    test = str(input("Enter the sentence for comparision: "))
    sentence = plagiarism_checker(source,test)
    # write a txt file with the contents
    with open('Results.txt', 'w') as writefile:
        writefile.write(sentence)
    menu()
  elif choice == 2:
    print("Upload the first file:\n")
    from google.colab import files
    uploaded = files.upload()
    filename1 = list(uploaded.keys())
    for file in filename1:
        source = uploaded[file].decode("utf-8")
    print("Upload the second file:\n")
    from google.colab import files
    uploaded = files.upload()
    filename2 = list(uploaded.keys())
    for file in filename2:
        test = uploaded[file].decode("utf-8")
    # to avoid conversion errors.
    source.replace("-"," ")
    test.replace("-"," ")
    sentence = plagiarism_checker(source,test)
    # write a txt file with the contents
    with open('Results.txt', 'w') as writefile:
      writefile.write(sentence)
    menu()
  elif choice == 3 :
    from google.colab import files
    makePDF("Results.txt")
    files.download('Results.pdf')
  elif choice == 4 :
    print("THANK YOU")
  else:
      print("You must only select either 1 2 3 or 4")
      menu()

In [ ]:
Test = "The First sentence is about Python. The Second: about Django. You can learn Python,Django and Data Analysis here."
Source = "The first is differenta. This second sentence is different. The third sentence is also very different."
Source2 = "The Initial sentence is not about Java. This second sentence is different. You can learn everything about anything here."
Source3 = "The Initial sentence is about Python. The Second: about Flask. You can learn Python,Django and Data manipulation here."

# **CASE 1:**
## First we will be testing same sentences against each other.
## This should result in a 100% plagarism.

In [ ]:
menu()

********WELCOME TO PLAGIARISM DETECTION TOOL**********


                      1: Sentence plagiarism checker
                      2: Document plagiarism checker
                      3: Download plagiarism report and Quit
                      4: Quit!
                      Please enter your choice: 1
Enter the sentence to be checked: The First sentence is about Python. The Second: about Django. You can learn Python,Django and Data Analysis here.
Enter the sentence for comparision: The first is differenta. This second sentence is different. The third sentence is also very different.

s1 is:  first sentence python
s2 is:  first differenta
Leven:  17
Max Len:  21

s1 is:  second: django
s2 is:  second sentence different
Leven:  21
Max Len:  25

s1 is:  learn python,django data analysis here.
s2 is:  third sentence also different.
Leven:  41
Max Len:  39
The document is 100% unique
********WELCOME TO PLAGIARISM DETECTION TOOL**********


                      1: Sentence plagiarism chec

# **CASE 2:**
## Now we will be testing two completely different sentences against each other.
## This should result in 100% unique.

In [ ]:
menu()

********WELCOME TO PLAGIARISM DETECTION TOOL**********


                      1: Sentence plagiarism checker
                      2: Document plagiarism checker
                      3: Download plagiarism report and Quit
                      4: Quit!
                      Please enter your choice: 1
Enter the sentence to be checked: The First sentence is about Python. The Second: about Django. You can learn Python,Django and Data Analysis here.
Enter the sentence for comparision: The first is differenta. This second sentence is different. The third sentence is also very different.
The document is 100% unique
********WELCOME TO PLAGIARISM DETECTION TOOL**********


                      1: Sentence plagiarism checker
                      2: Document plagiarism checker
                      3: Download plagiarism report and Quit
                      4: Quit!
                      Please enter your choice: 4
THANK YOU


# **CASE 3:**
## Now we will be testing two sentences ,which are having some similarity ,against each other.

In [ ]:
menu()

********WELCOME TO PLAGIARISM DETECTION TOOL**********


                      1: Sentence plagiarism checker
                      2: Document plagiarism checker
                      3: Download plagiarism report and Quit
                      4: Quit!
                      Please enter your choice: 1
Enter the sentence to be checked: The First sentence is about Python. The Second: about Django. You can learn Python,Django and Data Analysis here.
Enter the sentence for comparision: The Initial sentence is not about Java. This second sentence is different. You can learn everything about anything here.
The document is 100% unique
********WELCOME TO PLAGIARISM DETECTION TOOL**********


                      1: Sentence plagiarism checker
                      2: Document plagiarism checker
                      3: Download plagiarism report and Quit
                      4: Quit!
                      Please enter your choice: 4
THANK YOU


# **CASE 4:**
## Now we will be testing two sentences , in which only some words are replaced by other words of similar meaning , against each other.

In [ ]:
menu()

********WELCOME TO PLAGIARISM DETECTION TOOL**********


                      1: Sentence plagiarism checker
                      2: Document plagiarism checker
                      3: Download plagiarism report and Quit
                      4: Quit!
                      Please enter your choice: 1
Enter the sentence to be checked: The First sentence is about Python. The Second: about Django. You can learn Python,Django and Data Analysis here.
Enter the sentence for comparision: The Initial sentence is about Python. The Second: about Flask. You can learn Python,Django and Data manipulation here.
Sentence 1 is plagiarised
Sentence 3 is plagiarised
The document has 66.66666666666666 % of plagiarism detected
********WELCOME TO PLAGIARISM DETECTION TOOL**********


                      1: Sentence plagiarism checker
                      2: Document plagiarism checker
                      3: Download plagiarism report and Quit
                      4: Quit!
                      Ple

## We now test two txt files in which the second one contains a paraphrased version of the contents from the first file.

In [ ]:
menu()

********WELCOME TO PLAGIARISM DETECTION TOOL**********


                      1: Sentence plagiarism checker
                      2: Document plagiarism checker
                      3: Download plagiarism report and Quit
                      4: Quit!
                      Please enter your choice: 2
Upload the first file:



## We now test two txt files in which are completely different to each other.

In [ ]:
menu()

********WELCOME TO PLAGIARISM DETECTION TOOL**********


                      1: Sentence plagiarism checker
                      2: Document plagiarism checker
                      3: Download plagiarism report and Quit
                      4: Quit!
                      Please enter your choice: 2
Upload the first file:



Saving case_2.txt to case_2.txt
Upload the second file:



Saving case_3.txt to case_3.txt
The document is 100% unique
********WELCOME TO PLAGIARISM DETECTION TOOL**********


                      1: Sentence plagiarism checker
                      2: Document plagiarism checker
                      3: Download plagiarism report and Quit
                      4: Quit!
                      Please enter your choice: 3


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>